# Binance Algo-Trading: Momentum Strategy for BTC/USDT
**Date**: July 31, 2025  
**Purpose**: Implements a momentum-based trading bot for Binance BTC/USDT, buying on high 5-hour returns (>5%) and volume (>20-hour average), selling after 24 hours or 5% stop-loss. Uses Binance API, Plotly, and VaR.  
**Framework**: Define Hypothesis, Collect Data, Build Logic, Backtest, Optimize/Validate, Deploy  
**Notes**: API keys in .env; project tracked on GitHub.  
**Setup**:
```bash
cd C:\Users\emcm\Desktop\Project_R\crypto_trading_project
mkdir notebooks data logs outputs outputs\plots
echo .env\ndata/\nlogs/ > .gitignore
pip install -U ccxt pandas numpy plotly python-dotenv kaleido
git init

In [2]:
# Imports and Setup
import ccxt
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import logging
from dotenv import load_dotenv
import os
import time

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(message)s",
    handlers=[
        logging.FileHandler("../logs/trading.log"),  # Fixed path
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Load API keys from .env
load_dotenv()
api_key = os.getenv("BINANCE_API_KEY")
api_secret = os.getenv("BINANCE_API_SECRET")

# Initialize Binance
if api_key and api_secret:
    exchange = ccxt.binance({
        "apiKey": api_key,
        "secret": api_secret,
        "enableRateLimit": True
    })
    logger.info("Binance API initialized with authentication.")
else:
    exchange = ccxt.binance({"enableRateLimit": True})
    logger.info("Binance API initialized (public access).")

# Test API connectivity
try:
    symbol = "BTC/USDT"
    ticker = exchange.fetch_ticker(symbol)
    print(f"Current BTC/USDT Price: ${ticker['last']:.2f}")
except Exception as e:
    logger.error(f"API test failed: {e}")

2025-07-31 02:37:40,711 - Binance API initialized (public access).


Current BTC/USDT Price: $117580.00


In [ ]:
# Data Collection: OHLCV for BTC/USDT
logger.info("Fetching BTC/USDT data...")
symbol = "BTC/USDT"
timeframe = "1h"
limit = 500

try:
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit)
    df = pd.DataFrame(ohlcv, columns=["timestamp", "open", "high", "low", "close", "volume"])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    
    # Clean data
    df = df.dropna()
    df = df[df["volume"] > 0]
    logger.info("Data fetched and cleaned!")
    df.to_csv("data/btc_usdt_data.csv", index=False)
    print(df.head())
except Exception as e:
    logger.error(f"Data fetch error: {e}")
    time.sleep(2)

In [ ]:
# Momentum Strategy Logic
class MomentumStrategy:
    """Momentum strategy: Buy on high returns and volume, sell after hold period or stop-loss."""
    def __init__(self, return_period=5, volume_period=20, min_return=0.05, hold_period=24):
        self.return_period = return_period
        self.volume_period = volume_period
        self.min_return = min_return
        self.hold_period = hold_period
    
    def generate_signals(self, df):
        """Generate buy signals based on returns and volume."""
        df["returns"] = df["close"].pct_change(periods=self.return_period)
        df["avg_volume"] = df["volume"].rolling(window=self.volume_period).mean()
        df["signal"] = 0
        df.loc[(df["returns"] > self.min_return) & (df["volume"] > df["avg_volume"]), "signal"] = 1
        return df

# Apply strategy
logger.info("Generating signals...")
strategy = MomentumStrategy()
df = pd.read_csv("data/btc_usdt_data.csv")
df["timestamp"] = pd.to_datetime(df["timestamp"])
df = strategy.generate_signals(df)

# Visualize with Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=df["timestamp"], y=df["close"], name="Close Price", line=dict(color="blue")))
fig.add_trace(go.Scatter(x=df["timestamp"], y=df["returns"] * 1000, name="5-Hour Returns (scaled)", line=dict(color="orange")))
buy_signals = df[df["signal"] == 1]
fig.add_trace(go.Scatter(x=buy_signals["timestamp"], y=buy_signals["close"], mode="markers", name="Buy Signal", marker=dict(symbol="triangle-up", size=10, color="green")))
fig.update_layout(title="BTC/USDT Momentum Strategy", xaxis_title="Time", yaxis_title="Price (USDT)", showlegend=True)

# Save plot with error handling
try:
    fig.write_image("outputs/plots/momentum_signals.png")
    logger.info("Plot saved to outputs/plots/momentum_signals.png")
except Exception as e:
    logger.error(f"Failed to save plot: {e}")
    print("Plot not saved; displaying only.")

fig.show()
print("Last 5 rows:")
print(df[["timestamp", "close", "returns", "avg_volume", "signal"]].tail())

In [ ]:
# Backtesting Momentum Strategy
def calculate_var(returns, confidence_level=0.95):
    """Calculate Value at Risk."""
    return np.percentile(returns, (1 - confidence_level) * 100)

# Backtest
logger.info("Running backtest...")
initial_balance = 10000
balance = initial_balance
position = 0
entry_price = 0
entry_time = None
trade_log = []
equity = [initial_balance]
returns = []
fee_rate = 0.001  # 0.1% Binance fee
stop_loss = 0.95  # 5% stop-loss
hold_period = 24  # 24 hours

for i in range(1, len(df)):
    signal = df["signal"].iloc[i]
    price = df["close"].iloc[i]
    timestamp = df["timestamp"].iloc[i]
    prev_signal = df["signal"].iloc[i-1]

    if position == 1:
        hours_held = (timestamp - entry_time).total_seconds() / 3600
        if price < entry_price * stop_loss or hours_held >= hold_period:
            position = 0
            ret = (price - entry_price) / entry_price
            balance *= (1 + ret) * (1 - fee_rate)
            returns.append(ret)
            reason = "Stop-Loss" if price < entry_price * stop_loss else "Hold Period"
            logger.info(f"{reason} Sell at {price:.2f}, Return: {ret:.4f}")
            trade_log.append(f"{reason} Sell at {price:.2f} on {timestamp}, Return: {ret:.4f}")
    
    elif signal == 1 and prev_signal != 1 and position == 0:
        position = 1
        entry_price = price
        entry_time = timestamp
        balance *= (1 - fee_rate)
        logger.info(f"Buy at {price:.2f}")
        trade_log.append(f"Buy at {price:.2f} on {timestamp}")
    
    equity.append(balance if position == 0 else balance * (price / entry_price))

# Calculate VaR
if returns:
    var = calculate_var(returns)
    logger.info(f"VaR (95% confidence): {var:.4f}")

# Plot equity curve
fig = go.Figure()
fig.add_trace(go.Scatter(x=df["timestamp"], y=equity[:len(df)], name="Portfolio Value", line=dict(color="purple")))
fig.update_layout(title="Portfolio Equity Curve", xaxis_title="Time", yaxis_title="Portfolio Value (USDT)")
try:
    fig.write_image("outputs/plots/equity_curve.png")
    logger.info("Equity curve saved to outputs/plots/equity_curve.png")
except Exception as e:
    logger.error(f"Failed to save equity curve: {e}")
    print("Equity curve not saved; displaying only.")
fig.show()

print(f"Initial Balance: ${initial_balance:.2f}")
print(f"Final Balance: ${balance:.2f}")
print(f"Return: {(balance - initial_balance) / initial_balance * 100:.2f}%")
print(f"Number of Trades: {len(trade_log) // 2}")
print("\nTrade Log:")
for log in trade_log:
    print(log)

In [ ]:
# Optimization and Validation
def backtest_strategy(df, strategy_params):
    """Run backtest with given strategy parameters."""
    strategy = MomentumStrategy(**strategy_params)
    df = strategy.generate_signals(df)
    
    initial_balance = 10000
    balance = initial_balance
    position = 0
    entry_price = 0
    entry_time = None
    returns = []
    trade_log = []
    fee_rate = 0.001
    stop_loss = 0.95
    hold_period = strategy_params["hold_period"]
    
    for i in range(1, len(df)):
        signal = df["signal"].iloc[i]
        price = df["close"].iloc[i]
        timestamp = df["timestamp"].iloc[i]
        prev_signal = df["signal"].iloc[i-1]
        
        if position == 1:
            hours_held = (timestamp - entry_time).total_seconds() / 3600
            if price < entry_price * stop_loss or hours_held >= hold_period:
                position = 0
                ret = (price - entry_price) / entry_price
                balance *= (1 + ret) * (1 - fee_rate)
                returns.append(ret)
        
        elif signal == 1 and prev_signal != 1 and position == 0:
            position = 1
            entry_price = price
            entry_time = timestamp
            balance *= (1 - fee_rate)
    
    total_return = (balance - initial_balance) / initial_balance * 100
    return {
        "return": total_return,
        "trades": len(trade_log) // 2,
        "var": calculate_var(returns) if returns else None
    }

# Optimization parameters
param_grid = [
    {"return_period": 5, "volume_period": 20, "min_return": mr, "hold_period": hp}
    for mr in [0.03, 0.05, 0.07]
    for hp in [12, 24, 48]
]

# Split data for training and validation
df = pd.read_csv("data/btc_usdt_data.csv")
df["timestamp"] = pd.to_datetime(df["timestamp"])
train_df = df.iloc[:250].copy()
test_df = df.iloc[250:].copy()

# Optimize on training data
logger.info("Running optimization...")
results = []
for params in param_grid:
    result = backtest_strategy(train_df, params)
    result.update(params)
    results.append(result)

# Display optimization results
results_df = pd.DataFrame(results)
print("\nOptimization Results (Training Data):")
print(results_df[["return_period", "volume_period", "min_return", "hold_period", "return", "trades", "var"]])

# Validate best parameters on test data
best_params = results_df.loc[results_df["return"].idxmax()]
logger.info(f"Best parameters: {best_params[['return_period', 'min_return', 'hold_period']]}")
validation_result = backtest_strategy(test_df, best_params)
print("\nValidation Results (Test Data):")
print(f"Return: {validation_result['return']:.2f}%")
print(f"Number of Trades: {validation_result['trades']}")
print(f"VaR (95% confidence): {validation_result['var']:.4f if validation_result['var'] is not None else 'N/A'}")

# Plot validation equity curve
strategy = MomentumStrategy(**best_params)
test_df = strategy.generate_signals(test_df)
balance = initial_balance = 10000
position = 0
entry_price = 0
entry_time = None
equity = [initial_balance]
fee_rate = 0.001
stop_loss = 0.95

for i in range(1, len(test_df)):
    signal = test_df["signal"].iloc[i]
    price = test_df["close"].iloc[i]
    timestamp = test_df["timestamp"].iloc[i]
    prev_signal = test_df["signal"].iloc[i-1]
    
    if position == 1:
        hours_held = (timestamp - entry_time).total_seconds() / 3600
        if price < entry_price * stop_loss or hours_held >= best_params["hold_period"]:
            position = 0
            ret = (price - entry_price) / entry_price
            balance *= (1 + ret) * (1 - fee_rate)
    
    elif signal == 1 and prev_signal != 1 and position == 0:
        position = 1
        entry_price = price
        entry_time = timestamp
        balance *= (1 - fee_rate)
    
    equity.append(balance if position == 0 else balance * (price / entry_price))

fig = go.Figure()
fig.add_trace(go.Scatter(x=test_df["timestamp"], y=equity[:len(test_df)], name="Portfolio Value", line=dict(color="purple")))
fig.update_layout(title="Validation Equity Curve (Test Data)", xaxis_title="Time", yaxis_title="Portfolio Value (USDT)")
try:
    fig.write_image("outputs/plots/validation_equity_curve.png")
    logger.info("Validation equity curve saved to outputs/plots/validation_equity_curve.png")
except Exception as e:
    logger.error(f"Failed to save validation equity curve: {e}")
    print("Validation equity curve not saved; displaying only.")
fig.show()